In [112]:
import os
import zipfile
import tempfile
import random
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pywintypes
import win32file
import win32con
import win32com.client

def losowa_data_utworzenia():
    start = datetime(2024, 12, 15)
    end = datetime(2025, 5, 31)
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def losowa_data_modyfikacji_wzgledem_utworzenia(data_utw):
    min_data = data_utw + timedelta(days=5)
    max_data = data_utw + timedelta(days=20)
    delta = max_data - min_data
    return min_data + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def losowa_data_drukowania():
    start = datetime(2025, 8, 1)
    end = datetime(2025, 12, 10)
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

def format_data_iso(dt):
    return dt.isoformat(timespec='seconds') + "Z"

def ustaw_systemowa_date_pliku(file_path, utw_dt, mod_dt):
    wintime_utw = pywintypes.Time(utw_dt)
    wintime_mod = pywintypes.Time(mod_dt)
    handle = win32file.CreateFile(
        file_path, win32con.GENERIC_WRITE, 0, None,
        win32con.OPEN_EXISTING, win32con.FILE_ATTRIBUTE_NORMAL, None)
    win32file.SetFileTime(handle, wintime_utw, None, wintime_mod)
    handle.close()

def modyfikuj_corexml(path, utw_dt, mod_dt, print_dt):
    ns = {
        'cp': 'http://schemas.openxmlformats.org/package/2006/metadata/core-properties',
        'dc': 'http://purl.org/dc/elements/1.1/',
        'dcterms': 'http://purl.org/dc/terms/',
        'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
    }
    tree = ET.parse(path)
    root = tree.getroot()

    def wyczysc(tag):
        el = root.find(tag, ns)
        if el is not None:
            el.text = ""

    wyczysc('dc:title')
    wyczysc('dc:subject')
    wyczysc('cp:manager')
    wyczysc('cp:company')

    if (el := root.find('dc:creator', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('cp:lastModifiedBy', ns)) is not None:
        el.text = "Anonim"
    if (el := root.find('dcterms:created', ns)) is not None:
        el.text = format_data_iso(utw_dt)
    if (el := root.find('dcterms:modified', ns)) is not None:
        el.text = format_data_iso(mod_dt)

    last_printed = root.find('cp:lastPrinted', ns)
    if last_printed is None:
        last_printed = ET.SubElement(root, '{http://schemas.openxmlformats.org/package/2006/metadata/core-properties}lastPrinted')
    last_printed.text = format_data_iso(print_dt)

    tree.write(path, encoding='utf-8', xml_declaration=True)

def modyfikuj_appxml(path):
    try:
        tree = ET.parse(path)
        root = tree.getroot()
        for tag in ['Company', 'Manager', 'Application', 'Title', 'Subject']:
            el = root.find(tag)
            if el is not None:
                el.text = ""
        tree.write(path, encoding='utf-8', xml_declaration=True)
    except:
        pass

def przetworz_xlsx(file_path, output_path, utw_dt, mod_dt, print_dt):
    with tempfile.TemporaryDirectory() as tempdir:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(tempdir)

        core_path = os.path.join(tempdir, 'docProps', 'core.xml')
        app_path = os.path.join(tempdir, 'docProps', 'app.xml')

        if os.path.exists(core_path):
            modyfikuj_corexml(core_path, utw_dt, mod_dt, print_dt)
        if os.path.exists(app_path):
            modyfikuj_appxml(app_path)

        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as new_zip:
            for foldername, subfolders, filenames in os.walk(tempdir):
                for filename in filenames:
                    full_path = os.path.join(foldername, filename)
                    rel_path = os.path.relpath(full_path, tempdir)
                    new_zip.write(full_path, rel_path)

    ustaw_systemowa_date_pliku(output_path, utw_dt, mod_dt)

def konwertuj_xls_na_xlsm(excel, file_path):
    output_path = file_path + "m"
    wb = excel.Workbooks.Open(file_path, False, False, None, None, None, None, None, None, True)  # Open as editable
    wb.SaveAs(output_path, FileFormat=52)  # 52 = .xlsm (z makrami)
    wb.Close()
    return output_path

def przetworz_folder_rekursywnie(folder_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.DisplayAlerts = False
    excel.Visible = False

    for root, dirs, files in os.walk(folder_path):
        for fname in files:
            full_path = os.path.join(root, fname)

            # Konwersja .xls → .xlsm (zachowując makra)
            if fname.endswith(".xls") and not fname.endswith(".xlsx"):
                try:
                    print(f"🔄 Konwertuję .xls → .xlsm: {fname}")
                    full_path = konwertuj_xls_na_xlsm(excel, full_path)
                    fname = os.path.basename(full_path)
                except Exception as e:
                    print(f"⚠️ Błąd konwersji .xls → .xlsm: {fname} → {e}")
                    continue

            if fname.endswith(".xlsx") or fname.endswith(".xlsm"):
                if "_random_date" in fname:
                    continue

                utw_dt = losowa_data_utworzenia()
                mod_dt = losowa_data_modyfikacji_wzgledem_utworzenia(utw_dt)
                print_dt = losowa_data_drukowania()

                new_name = fname.replace(".xlsx", "_random_date.xlsx").replace(".xlsm", "_random_date.xlsm")
                output_path = os.path.join(root, new_name)

                try:
                    przetworz_xlsx(full_path, output_path, utw_dt, mod_dt, print_dt)
                    print(f"✅ {fname} → {os.path.basename(output_path)}")
                    print(f"   Utworzono: {utw_dt.date()} | Zmodyfikowano: {mod_dt.date()} | Drukowano: {print_dt.date()}")
                except Exception as e:
                    print(f"❌ Błąd przetwarzania {fname}: {e}")

    excel.Quit()


# 🔽 Ścieżka główna (można podać dowolną lokalizację)
folder = r"C:\Users\a\Desktop\directory_with_changed_files"
przetworz_folder_rekursywnie(folder)


✅ 2022 - EXP_expenses ••• Morocco.xlsx → 2022 - EXP_expenses ••• Morocco_random_date.xlsx
   Utworzono: 2025-01-10 | Zmodyfikowano: 2025-01-16 | Drukowano: 2025-11-24
✅ 2024 - EXP_expenses ••• Namibia.xlsx → 2024 - EXP_expenses ••• Namibia_random_date.xlsx
   Utworzono: 2025-01-06 | Zmodyfikowano: 2025-01-26 | Drukowano: 2025-12-05
✅ corax.xlsx → corax_random_date.xlsx
   Utworzono: 2025-03-31 | Zmodyfikowano: 2025-04-15 | Drukowano: 2025-08-21
✅ 2017 - EXP_emails ••• Puna de Atacama ••• people met on the road.xlsx → 2017 - EXP_emails ••• Puna de Atacama ••• people met on the road_random_date.xlsx
   Utworzono: 2025-01-10 | Zmodyfikowano: 2025-01-24 | Drukowano: 2025-09-08
✅ 2019 - EXP_expenses ••• Thailand & Myanmar.xlsx → 2019 - EXP_expenses ••• Thailand & Myanmar_random_date.xlsx
   Utworzono: 2025-04-03 | Zmodyfikowano: 2025-04-11 | Drukowano: 2025-12-01
